In [78]:
#-*- coding: utf-8 -*-
#csv파일 받아서 형태소화 시키고 csv파일로만들어줄거

In [79]:
from gensim.models.word2vec import Word2Vec
import glob
from konlpy.tag import Twitter 
import re
import pandas as pd

In [80]:
tagger = Twitter()

def data_read(data_place):

    f = open(data_place,'r')
    txt = f.read()

    hangle = re.compile('[^ ㄱ-ㅣ가-힣0-9]+')
    result = hangle.sub('',txt)
    #print("result clear")
    result = tagger.pos(result,norm=True,stem = True)
    return result
    
    
def train_data_make(data):
    r = []
    results = []
    jung_bok = ""
    for (word, pumsa) in data:
        if not pumsa in ["Josa", "Eomi", "Punctuation"] and jung_bok !=word:
            r.append(word)
            jung_bok = word
    results.append((" ".join(r)).strip())  
    return results
def train_data_to_txt(results,name = ''):
    with open(name,'w') as fp:
        fp.write(results)
        
def train_data_creation(data_dir,data_name):
    for i in range(len(data_name)):
        result = data_read(data_name[i])
        result = train_data_make(result)
        dir_name = data_dir+str(i)+'.txt'
        train_data_to_txt(result[0],dir_name)
        print(i,"/",len(data_name))

In [81]:
def tagging(one_row):
    hangle = re.compile('[^ ㄱ-ㅣ가-힣0-9]+')
    result = hangle.sub('',one_row)
    result = tagger.pos(result,norm=True,stem = True)
    return result

def data_preprocessing(one_row):
    data = tagging(one_row)
    r = []
    results = []
    jung_bok = ""
    for (word, pumsa) in data:
        if not pumsa in ["Josa", "Eomi", "Punctuation",'Number'] and jung_bok !=word:
            r.append(word)
            jung_bok = word
    results.append((" ".join(r)).strip())  
    return results[0]

def data_making(input_data):
    data = pd.read_csv(input_data,encoding='euc-kr').iloc[:,1:]
    data['times'] =data['times'].str[:10]
    data['times'] = data['times'].apply(lambda x : x.replace('.',"-"))
    data['str'] = data['titles'].str[:]+data['articles']
    del data['titles']
    del data['articles']
    data['str'] = data['str'].apply(data_preprocessing)
    return data

In [82]:
model = Word2Vec.load("./model/last_train_model.model")

In [119]:
male_data = glob.glob("important/male/csv/*.csv")

In [118]:
korea_data = glob.glob("./important/korea/csv/*.csv")

In [149]:
def dataframe_preprocessing(data,i):
    data = data_making(data)
    #print("data's shape",data.shape)
    data = data.iloc[i,1].split()
    data = word_to_vec(data)
    return data
    
def word_to_vec(data):
    word_vec_list = []
    for txt in data:
        try:
            vec = model.wv[txt]
            word_vec_list.append(vec)
        except:
            pass
    return word_vec_list

In [150]:
max_len = 0

In [ ]:
(dataframe_preprocessing(male_data[1],0))

In [151]:
for i in range(106):
    if( max_len < len(dataframe_preprocessing(male_data[1],i))):
        max_len = len(dataframe_preprocessing(male_data[1],i))
print(max_len)

1722
